In [9]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost
import xgboost as xgb
import pandas as pd
import shap
import joblib
import os
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from customer_churn_pridiction.config import PROCESSED_DATA_DIR, MLFLOW_DIR, MODELS_DIR, SHAP_PLOTS_DIR

In [2]:
train_data = pd.read_csv(PROCESSED_DATA_DIR / 'train.csv')
test_data = pd.read_csv(PROCESSED_DATA_DIR / 'test.csv')
target = 'Churn'

X_train = train_data.drop(columns=[target])
Y_train = train_data[target]
X_test = test_data.drop(columns=[target])
Y_test = test_data[target]

In [3]:
standard_scaler = joblib.load(PROCESSED_DATA_DIR / 'standard_scaler.joblib')
one_hot_encoder = joblib.load(PROCESSED_DATA_DIR / 'one_hot_encoder.joblib')

In [4]:

X_train_encoded = one_hot_encoder.transform(X_train.select_dtypes(include=['object']))

X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=one_hot_encoder.get_feature_names_out(X_train.select_dtypes(include=['object']).columns.tolist()))

X_train = pd.concat([X_train.select_dtypes(exclude=['object']), X_train_encoded_df], axis=1)

In [5]:
X_test_encoded = one_hot_encoder.transform(X_test.select_dtypes(include=['object']))

X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=one_hot_encoder.get_feature_names_out(X_test.select_dtypes(include=['object']).columns.tolist()))


X_test = pd.concat([X_test.select_dtypes(exclude=['object']), X_test_encoded_df], axis=1)

In [10]:
mlflow.set_tracking_uri(MLFLOW_DIR)
mlflow.set_experiment('customer_churn_prediction_experiment')

models = [
    ('XGBoost', xgb.XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=0)),
    ('RandomForest', RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)),
    ('LogisticRegression', LogisticRegression(max_iter=100, random_state=0))
]

# Loop through models
for model_name, model in models:
    with mlflow.start_run(run_name=model_name):
        # Fit the model and log parameters
        model.fit(standard_scaler.transform(X_train), Y_train)
        mlflow.log_param('model', model_name)

        # Log parameters specific to the model
        model_params = {
            'XGBoost': {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.1},
            'RandomForest': {'n_estimators': 100, 'max_depth': 5},
            'LogisticRegression': {'max_iter': 100}
        }
        for param, value in model_params[model_name].items():
            mlflow.log_param(param, value)

        # Predictions and accuracy
        Y_train_pred = model.predict(standard_scaler.transform(X_train))
        Y_test_pred = model.predict(standard_scaler.transform(X_test))
        train_accuracy = accuracy_score(Y_train, Y_train_pred)
        test_accuracy = accuracy_score(Y_test, Y_test_pred)
        
        mlflow.log_metric('train_accuracy', train_accuracy)
        mlflow.log_metric('test_accuracy', test_accuracy)

        # Log model and artifacts
        mlflow.sklearn.log_model(model, 'model')
        mlflow.log_artifact(PROCESSED_DATA_DIR / 'standard_scaler.joblib')
        mlflow.log_artifact(PROCESSED_DATA_DIR / 'one_hot_encoder.joblib')

        # Print results
        print(f'\nModel: {model_name}')
        print(f'Train accuracy: {train_accuracy:.4f}')
        print(f'Test accuracy: {test_accuracy:.4f}')
        print('Classification report for test data:')
        print(classification_report(Y_test, Y_test_pred))
        
        # SHAP explanations for tree-based models only
        if model_name in ['XGBoost', 'RandomForest']:
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(standard_scaler.transform(X_test))
            
            try:
                # Summary plot (bar)
                plt.figure()
                shap.summary_plot(shap_values, X_test, plot_type='bar')
                summary_plot_path = os.path.join(SHAP_PLOTS_DIR, f'shap_summary_plot_{model_name}.png')
                plt.savefig(summary_plot_path)
                plt.close()  # Close the figure after saving

                # Dependence plot
                plt.figure()
                shap.dependence_plot('Tenure', shap_values, X_test)
                dependence_plot_path = os.path.join(SHAP_PLOTS_DIR, f'shap_dependence_plot_tenure_{model_name}.png')
                plt.savefig(dependence_plot_path)
                plt.close()  # Close the figure after saving
                
                # Waterfall plot for a single instance
                plt.figure()
                shap.waterfall_plot(shap.Explanation(values=shap_values[0], base_values=explainer.expected_value, data=X_test.iloc[0]))
                waterfall_plot_path = os.path.join(SHAP_PLOTS_DIR, f'shap_waterfall_plot_{model_name}.png')
                plt.savefig(waterfall_plot_path)
                plt.close()  # Close the figure after saving

                # Beeswarm plot
                plt.figure()
                shap.summary_plot(shap_values, X_test, plot_type='dot')
                beeswarm_plot_path = os.path.join(SHAP_PLOTS_DIR, f'shap_beeswarm_plot_{model_name}.png')
                plt.savefig(beeswarm_plot_path)
                plt.close()  # Close the figure after saving
                
                # Force plot (specific data point)
                force_plot_path = os.path.join(SHAP_PLOTS_DIR, f'shap_force_plot_{model_name}.html')
                shap.force_plot(explainer.expected_value, shap_values[0], X_test.iloc[0], show=False)
                plt.close()  # Close any opened figures (if applicable)

                mlflow.log_artifact(force_plot_path)

            except Exception as e:
                print(f"Error logging SHAP plots for {model_name}: {e}")

        # Dump model after training
        model_dump_path = f"{model_name}_model.joblib"
        joblib.dump(model, MODELS_DIR / model_dump_path)
        mlflow.log_artifact(model_dump_path)

    print(f'Model: {model_name} - completed and logged all SHAP plots and model artifacts.')

2024/10/31 00:25:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
/opt/homebrew/Caskroom/miniconda/base/envs/customer-churn-prediction-pipeline/lib/python3.9/site-packages/shap/plots/_beeswarm.py:952: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  pl.show()



Model: XGBoost
Train accuracy: 0.9727
Test accuracy: 0.9383
Classification report for test data:
              precision    recall  f1-score   support

           0       0.95      0.98      0.96       898
           1       0.88      0.72      0.79       172

    accuracy                           0.94      1070
   macro avg       0.91      0.85      0.88      1070
weighted avg       0.94      0.94      0.94      1070



/opt/homebrew/Caskroom/miniconda/base/envs/customer-churn-prediction-pipeline/lib/python3.9/site-packages/shap/plots/_scatter.py:777: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  pl.show()
/opt/homebrew/Caskroom/miniconda/base/envs/customer-churn-prediction-pipeline/lib/python3.9/site-packages/shap/plots/_waterfall.py:315: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/opt/homebrew/Caskroom/miniconda/base/envs/customer-churn-prediction-pipeline/lib/python3.9/site-packages/shap/plots/_beeswarm.py:952: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  pl.show()


Error logging SHAP plots for XGBoost: [Errno 2] No such file or directory: '/Users/saadkhalid/Documents/epita/s2/ai_methodology/customer-churn-prediction-pipeline/shap/shap_force_plot_XGBoost.html'


FileNotFoundError: [Errno 2] No such file or directory: 'XGBoost_model.joblib'